In [2]:
import pandas as pd
import os
import shutil
import glob
import numpy as np
import csv
from pathlib import Path
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


In [3]:
mic_dframe = pd.read_csv('./Input/dados_teste/MIC_modelo.csv')

antibiotics = mic_dframe.columns[-18:]
antibiotics = antibiotics.to_list()
antibiotics.pop(2)
antibiotics.pop(4)
antibiotics.pop(5)
antibiotics.pop(9)
antibiotics.pop(12)

'tobramycin'

In [4]:


def sort_csv_files(source_dir, output_base_dir, num_folders=16):
    """
    Sort CSV files from source directory into a specified number of folders.
    
    Args:
        source_dir (str): Directory containing CSV files
        output_base_dir (str): Base directory for output folders
        num_folders (int): Number of folders to create
    """
    # Create output directories if they don't exist
    output_dirs = []
    for i in range(num_folders):
        folder_name = f"folder_{i+1}"
        folder_path = os.path.join(output_base_dir, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        output_dirs.append(folder_path)
    
    # Get list of all CSV files
    csv_files = glob.glob(os.path.join(source_dir, "*.csv"))
    
    # Verify files were found
    total_files = len(csv_files)
    if total_files == 0:
        print("No CSV files found in the source directory.")
        return
    
    print(f"Found {total_files} CSV files to distribute.")
    
    # Calculate files per folder
    files_per_folder = total_files // num_folders
    remainder = total_files % num_folders
    
    # Distribute files evenly across folders
    start_idx = 0
    for i, folder in enumerate(output_dirs):
        # Add one extra file to the first 'remainder' folders if division isn't even
        extra = 1 if i < remainder else 0
        end_idx = start_idx + files_per_folder + extra
        
        # Get the batch of files for this folder
        folder_files = csv_files[start_idx:end_idx]
        
        # Copy each file to the destination folder
        for file_path in folder_files:
            file_name = os.path.basename(file_path)
            dest_path = os.path.join(folder, file_name)
            shutil.move(file_path, dest_path)
            
        print(f"Copied {len(folder_files)} files to {folder}")
        start_idx = end_idx

In [4]:
source_directory = "./Input/dados_teste/Kmers 10/counts_k10_csv"
output_directory = "./Input/dados_teste/Separated_K10"
    
sort_csv_files(source_directory, output_directory)

No CSV files found in the source directory.


In [5]:

def csv_to_nparray(path):
    values = []
    with open(path, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            values.append([int(row[0])])
        f.close()
    
    return np.asarray(values)


def txt_to_nparray(path):
    values = []
    f = open(path, "r")
    for line in f:
        values.append(str(line))
    f.close()
    return np.asarray(values)



def best_N_features(target_df, antibiotic, N):
    genome_ids = target_df['Gene_ID'].loc[pd.notnull(target_df[antibiotic])].values
    folder_path = './Input/dados_teste/Separated_K10/folder_1/'
    final_list = []
    for csv_path in genome_ids:
        csv_path = csv_path + '.csv'
        temp_path = os.path.join(folder_path, csv_path)
        result = csv_to_nparray(temp_path)
        final_list.append(result)
    X = np.asarray(final_list)
    y = target_df[antibiotic].loc[pd.notnull(target_df[antibiotic])].values
    n_best = SelectKBest(chi2, k=N)
    X_new = n_best.fit_transform(X, y*10**5)
    kmers = np.load('./Input/dados_teste/Kmers 10/combinations_10.txt')
    selected_kmers = []
    n_best_selected = n_best.get_support()
    for i in range(0, len(n_best_selected)):
        if n_best_selected[i]:
            selected_kmers.append(kmers[i])
    best_feature_df = pd.DataFrame(X_new, columns = selected_kmers)
    
    return best_feature_df


def make_X(folder_path):
    genome_ids = os.listdir(folder_path)
    final_list = []
    count = 0
    for csv_path in genome_ids:
        genome_aux, _ = csv_path.split(".csv")
        if genome_aux in mic_dframe['Gene_ID'].values:
            temp_path = os.path.join(folder_path, csv_path)
            result = csv_to_nparray(temp_path)
            final_list.append(result)
            count += 1
            print(count)
            print(final_list)
    print("Finished part 1")
    X = np.asarray(final_list)
    return X


def best_N_features_modified(target_df, antibiotic, N, folder_path, X):
    genome_ids = os.listdir(folder_path)
    for gene in range(len(genome_ids)):
        genome_ids[gene], _ = genome_ids[gene].split(".csv")
    locs = []
    for i in range(mic_dframe.shape[0]):
        if mic_dframe['Gene_ID'][i] in genome_ids:
            locs.append(i)
    y = target_df[antibiotic].loc[locs].values
    X = X[~np.isnan(y)]
    y = y[~np.isnan(y)]
    n_best = SelectKBest(chi2, k=N)
    X_new = n_best.fit_transform(X, y*10**5)
    kmers = txt_to_nparray('./Input/dados_teste/Kmers 10/combinations_10.txt')
    selected_kmers = []
    n_best_selected = n_best.get_support()
    print("Entered part 2")
    for i in range(0, len(n_best_selected)):
        if n_best_selected[i]:
            selected_kmers.append(kmers[i])
    best_feature_df = pd.DataFrame(X_new, columns = selected_kmers)
    print("Finished part 2")
    
    return best_feature_df

In [6]:
folder_path = './Input/dados_teste/Separated_K10/folder_12/'

X_aux = make_X(folder_path)

1
[array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])]
2
[array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])]
3
[array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])]
4
[array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])]
5
[array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]]), array([[0],
       [0],
       [0],
   

KeyboardInterrupt: 

In [6]:
kmer_dframe = best_N_features_modified(target_df=mic_dframe, antibiotic=antibiotics[0], N=200, folder_path=folder_path, X=X_aux)

X = kmer_dframe.values

ValueError: Found array with dim 3. SelectKBest expected <= 2.